<a href="https://colab.research.google.com/github/Te1on/RuBART/blob/main/NewsSummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import dataset from kaggle

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "neuark"
os.environ['KAGGLE_KEY'] = "69df6862aa557c248cf0632c599e0228"

In [ ]:
!kaggle datasets download yutkin/corpus-of-russian-news-articles-from-lenta

 98% 569M/584M [00:08<00:00, 81.8MB/s]
100% 584M/584M [00:08<00:00, 72.5MB/s]


In [ ]:
!unzip "/content/corpus-of-russian-news-articles-from-lenta.zip"

Archive:  /content/corpus-of-russian-news-articles-from-lenta.zip
  inflating: lenta-ru-news.csv       


# Install Requirements

In [ ]:
!pip install transformers 

     |████████████████████████████████| 757kB 2.7MB/s 
     |████████████████████████████████| 3.0MB 9.0MB/s 
     |████████████████████████████████| 890kB 33.2MB/s 
     |████████████████████████████████| 1.1MB 41.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=9236785b3ab2a7e0f34872bf8f86d6e06099af0abe1ac75243733ef2e5eb89ff
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Main



In [ ]:
import pandas as pd
data = pd.read_csv('/content/lenta-ru-news.csv', usecols=['title','text'], skiprows=range(1,6))
data.head()

,title,text
0,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...
1,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...
2,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...
3,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...
4,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...


In [ ]:
len(data)

800970

In [ ]:
with open('news-text.txt','w') as f:
    for x in data['text']:
        f.write(str(x))

In [ ]:
import numpy as np
pool = 100
bk = int(np.ceil(len(data)/pool))
for i in range(pool):
    filename = "/content/drive/My Drive/RuBART/texts/news-texts[" + str(i) + "].txt"
    with open(filename,'w') as f:
        for text in data['text'][i:i+bk]:
            f.write(str(text) + '\n')

Training tokenizer

In [ ]:
import torch

from transformers import BartTokenizer, BartForConditionalGeneration
from IPython.display import display, Markdown

torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch_device)

cuda


In [ ]:
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files='/content/news-text.txt', show_progress=True, vocab_size=80000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>"
])

In [ ]:
tokenizer.save_model('/content/drive/My Drive/RuBART/RuBART-Lenta')

['/content/drive/My Drive/RuBART/RuBART-Lenta/vocab.json',
 '/content/drive/My Drive/RuBART/RuBART-Lenta/merges.txt']

Shorten text news - 256 token max

In [ ]:
mask = []
for text in data['text']:
    mask.append(len(tokenizer(str(text)).input_ids) <= 256)

In [ ]:
len(mask)

800970

In [ ]:
pd.Series(mask).value_counts()

True     491675
False    309295
dtype: int64

In [ ]:
short_data = data.loc[mask]

In [ ]:
len(short_data)

491675

In [ ]:
short_data.to_csv("/content/drive/My Drive/RuBART/news-short.csv", index=False)

Training BART model

In [ ]:
from transformers import BartTokenizerFast
tokenizer = BartTokenizerFast.from_pretrained("/content/drive/My Drive/RuBART/RuBART-Lenta")

In [ ]:
a = tokenizer("Съешь ещё этих мягких французских булочек")
for i in a.input_ids:
    print(tokenizer.decode([i]))

<s>
С
ъ
ешь
 ещё
 этих
 мягких
 французских
 бул
очек
</s>


In [ ]:
from transformers import BartConfig

config = BartConfig(
    vocab_size=80000,
    ctivation_dropout=0.0,
    extra_pos_embeddings=2,
    activation_function="gelu",
    d_model=768,
    encoder_ffn_dim=768,
    encoder_layers=12,
    encoder_attention_heads=12,
    decoder_ffn_dim=768,
    decoder_layers=12,
    decoder_attention_heads=16,
    encoder_layerdrop=0.0,
    decoder_layerdrop=0.0,
    attention_dropout=0.0,
    dropout=0.1,
    max_position_embeddings=256,
    init_std=0.02,
    is_encoder_decoder=True,
    normalize_before=False,
    add_final_layer_norm=False,
    scale_embedding=False,
    normalize_embedding=True,
    static_position_embeddings=False,
    add_bias_logits=False,
)

In [ ]:
from transformers import BartForConditionalGeneration

model = BartForConditionalGeneration(config=config)

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/drive/My Drive/RuBART/news-short.csv",
    block_size=256
)

In [ ]:
tokenized = list(map(lambda x: tokenizer(str(x)).input_ids, data['text']))
tokens_length = list(map(lambda x: len(x), tokenized))

In [ ]:
pd.Series(ar_len).describe()

count    8105.000000
mean      209.467859
std       100.314393
min         4.000000
25%       137.000000
50%       191.000000
75%       259.000000
max       512.000000
dtype: float64

In [ ]:
dataset[0]

tensor([    0,  7088,  1610,  1834,  4487,  6467, 78407,  3369,  8125,  8981,
           16,  3780,  9102, 15791,   309, 15483,   375, 12876,     6,  2395,
          938, 47738,  5888,   389,    17,  2224,  6925,    18, 75742,  3428,
        53861,  1515,    30,   375,  2774,  5465, 49506,  1858,  6401,    16,
          948, 44206,   362,    16,   373, 16568, 15483, 26367,   281,  6674,
          507,  6349,  9084, 41562, 31691, 22520, 15364,   290,   825,    16,
          373,  9193,  2794,   375, 12876,     6, 19681,    16,   948, 79776,
          349, 75896,  2814,   457,   938,   281, 27480, 18131,    16, 54426,
          309, 38823,   616, 60378,  2460,    18, 12790, 12524,    16,   373,
         2543, 64913, 37837, 55079,   309,   375, 12876,     6,   281,  3442,
         1131,  3145,  4183,   488,    18, 10178, 41337,  2690, 11287, 15949,
        15483,   309, 37849, 11285,    16,   767,  2446, 11658,    17,  2811,
          470,   375, 12876,     6,   281,  5240,   284, 19889, 

In [ ]:
print(tokenizer.decode(list(dataset[5])))

<s>С 1 сентября на всей территории России вводится новая форма таможенной декларации для физических лиц. Как пояснили в Государственном таможенном комитете (ГТК), форма декларации, которая была одобрена Советом руководителей таможенных служб государств-участников Содружества, является единой на территории всех стран СНГ, за исключением Армении. По данным ИТАР-ТАСС, в новой декларации содержится более подробный перечень товаров, подлежащих обязательному декларированию. Так, впервые в их число включены ядовитые и сильно действующие вещества и лекарства, радиоактивные материалы, объекты флоры и фауны, а также высокочастотные радио-электронные устройства и средства связи - в частности, системы радиолокации, кабельного телевидения и средства мобильной связи частотой выше 9 килогерц. Кроме того, в декларации будет необходимо приводить более подробные сведения о ввозимом или вывозимом транспортном средстве - вид машинs, год выпуска, объем двигателя, номера двигателя, кузова и шасси. В России 

In [ ]:
import pandas as pd
import numpy as np
ar_len = [len(sent) for sent in dataset]
pd.Series(ar_len).describe()

count    8105.000000
mean      209.466872
std       100.313788
min         4.000000
25%       137.000000
50%       191.000000
75%       259.000000
max       512.000000
dtype: float64

In [ ]:
np.quantile(ar_len,0.98)

505.9200000000001

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/RuBART-Lenta",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("/content/drive/My Drive/RuBART/RuBART-Lenta")

Cheking model

In [ ]:
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    model="/content/drive/My Drive/RuBART/RuBART-Lenta",
    tokenizer="/content/drive/My Drive/RuBART/RuBART-Lenta",
)

In [ ]:
summarizer("""ЦИК обработал 100% протоколов. Против поправок выступили 21,27% россиян. 
Поправки к Конституции РФ на общероссийском голосовании поддержали 77,92% проголосовавших, следует из данных Центральной избирательной комиссии (ЦИК) РФ по итогам обработки 100% протоколов.
Согласно данным, опубликованным на экране в информационном центре ЦИК, против поправок к конституции выступили 21,27% проголосовавших. 
Общая явка, по последним данным, составила 65%.
Общероссийское голосование по поправкам к Конституции РФ завершилось на всей территории России в 21:00 мск в среду. 
""")